In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import warnings
from sklearn import metrics
import os

In [3]:
warnings.filterwarnings('ignore')

In [7]:
# 批量读取和合并文本数据集

def read_text(path,text_list):
    """
    :param path: 必选参数，文件夹路径
    :param text_list: 必选参数，文件夹 path 下的所有 .txt 文件名
    :return:返回值
            features 文本特征数据，以列表形式返回
            labels 分类标签，以列表形式返回
    """
    features, labels = [], []
    for text in text_list:
        if text.split('.')[-1] == 'txt':
            try:
                with open(path + text, encoding='gbk') as fp:
                    features.append(fp.read()) # 特征
                    labels.append(path.split('\\')[-2]) # 标签
            except Exception as erro:
                print('\n>>>发现错误，正在输出错误信息。。。\n', erro)
    return features, labels


def merge_text(train_or_text, label_name):
    """

    :param train_or_text: 必选参数，train 训练数据集 or text 测试数据集
    :param label_name: 必选参数，分类标签的名字
    :return: 返回值
            merge_features 合并好的所有特征数据，以列表形式返回
            merge_labels 合并好的所有分类标签数据，以列表形式返回
    """
    print('\n>>>文本读取和合并程序已经启动，请稍等。。。。')
    merge_features, merge_labels = [], []
    for name in label_name:
        path = 'D:\\NLP自然语言处理\\text_classification\\text classification\\' + train_or_text + '\\' + name + '\\'
        # print(path)
        text_list = os.listdir(path)
        # print(text_list)
        features, labels = read_text(path=path, text_list=text_list)
        merge_features += features # 特征
        merge_labels += labels # 标签
    return merge_features, merge_labels

In [ ]:
# 获取训练集
train_or_text = 'train'
label_name = ['女性','体育','校园','文学']
X_train, y_train = merge_text(train_or_text, label_name)
# 获取测试集
train_or_text = 'test'
label_name = ['女性','体育','校园','文学']
X_test, y_test = merge_text(train_or_text, label_name)


### 中文文本分词

In [12]:
# 训练集
import jieba
X_train_word = [jieba.cut(words) for words in X_train]
X_train_cut = [' '.join(word) for word in X_train_word]
X_train_cut[:5]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\azhen\AppData\Local\Temp\jieba.cache
Loading model cost 0.595 seconds.
Prefix dict has been built successfully.


['明天 三里屯 见 瑞丽 服饰 美容 ： 2011 瑞丽 造型 大赏 派对 瑞丽 专属 模特 黄美熙 康猴 猴 康乐 帕丽扎 提 也 会 参加 哦 瑞丽 专属 模特 转发 ( 53 ) 评论 ( 5 ) 12 月 8 日 17 : 10 来自 新浪 微博',
 '转发 微博 水草 温 ： 北京 早安 中国 宝迪沃 减肥 训练营 2012 年 寒假 班 开始 报名 早安 中国 吧 : 51 （ 分享 自 百度 贴 吧 ） 您 还 在 为 肥胖 为难 吗 冬季 减肥 训练 帮助 您 快速 减掉 多余 脂肪 咨询 顾问 温 老师 1071108677 原文 转发 原文 评论',
 '多谢 支持 ， 鱼子 精华 中 含有 太平洋 深海 凝胶 能 给 秀发 源源 滋养 ， 鱼子 精华 发膜 含有 羟基 积雪草 苷 ， 三重 神经酰胺 能 补充 秀发 流失 的 胶质 ， 可以 有 焕然 新生 的 感觉 。 鲍艺芳 ： 我 在 倍 丽莎 美发 沙龙 做 的 奢华 鱼子酱 护理 超级 好用 以后 就 用 它 了 我 的 最 爱 巴黎 卡诗 原文 转发 原文 评论',
 '注射 除皱 针 时会 疼 吗 ？ 其他 产品 注射 会 产生 难以忍受 的 疼痛 ， 除皱 针中 含有 微量 的 利多卡因 以 减轻 注射 过程 中 的 疼痛 ， 注射 过程 只有 非常 轻微 的 疼痛感 。 另外 ， 除皱 针 的 胶原蛋白 具有 帮助 凝血 作用 ， 极少 产生 淤血 现象 。 一般 注射 后 就 可以 出门 。 其他 产品 容易 产生 淤斑 ， 而且 时间 较长 。 让 人 难以 接受 。',
 '注射 除皱 针 时会 疼 吗 ？ 其他 产品 注射 会 产生 难以忍受 的 疼痛 ， 除皱 针中 含有 微量 的 利多卡因 以 减轻 注射 过程 中 的 疼痛 ， 注射 过程 只有 非常 轻微 的 疼痛感 。 另外 ， 除皱 针 的 胶原蛋白 具有 帮助 凝血 作用 ， 极少 产生 淤血 现象 。 一般 注射 后 就 可以 出门 。 其他 产品 容易 产生 淤斑 ， 而且 时间 较长 。 让 人 难以 接受 。']

In [13]:
# 测试集
import jieba
X_test_word = [jieba.cut(words) for words in X_test]
X_test_cut = [' '.join(word) for word in X_test_word]
X_test_cut[:5]

['【 L 丰胸 — — 安全 的 丰胸 手术 】 华美 L 美胸 非常 安全 ， 5 大 理由 ： ① 丰胸 手术 在世界上 已有 超过 100 年 的 历史 ; ② 假体 硅胶 有 40 多年 的 安全 使用 史 ; ③ 每年 都 有 超过 600 万 女性 接受 丰胸 手术 ; ④ 不 损伤 乳腺 组织 ， 不 影响 哺乳 ; ⑤ 华美 美莱 率先 发布 中国 首部 《 整形 美容 安全 消费 白皮书 》 ， 维护 社会 大众 整形 美容 消费 利益 。 \t \t \t \t $ LOTOzf $',
 '【 穿 高跟鞋 不磨 脚 小 方法 】 1 . 用 热 毛巾 在 磨 脚 的 部位 捂 几分钟 , 再拿块 干 的 软 毛巾 垫 着 , 用 锤子 把 鞋子 磨脚 的 地方 敲 平整 就 可以 啦 ； 2 . 把 报纸 捏 成团 沾点 水 , 不要 太湿 , 但 要 整团 都 沾 到 水 , 再 拿 张干 的 报纸 裹住 湿 的 报纸 , 塞 在 挤 脚 部位 , 用 塑料袋 密封 一夜 ； 3 . 穿鞋 之前 , 拿 香皂 ( 蜡烛 亦可 ) 在 磨脚 位置 薄薄的 涂上一层 。 \t \t \t \t $ LOTOzf $',
 '要 在 有限 的 空间 里 实现 最大 的 价值 ， 布 上网 线 、 音频线 、 高清 线 、 投影 线 、 电源线 、 闭 路线 等 一切 能够 想到 的 线 ； 在 32 ㎡ 里 放 一套 沙发 和 预留 5 - 6 个人 的 办公 环境 ； 5 ㎡ 里 做 一个 小 卧室 ； 在 6 ㎡ 里装 一个 袖珍 厨房 和 一个 厕所 ； 还要 在 2 米 * 5 米 的 阳台 上 做 一个 小 花园 ； 除 阳台 外套 内 只有 43 ㎡ 的 房子 要 怎么 做 才能 合理 利用 ？ $ LOTOzf $',
 '【 女人 命好 的 几点 特征 】   1 . 圆脸 ( 对 人际关系 及 财运 都 有加 分 ) ； 2 . 下巴 丰满 （ 可能 会 拥有 两栋 以上 的 房产 ） ； 3 . 臀大 ( 代表 有 财运 ) ； 4 . 腿 不能 细 （ 腿 长脚 瘦 ， 奔走 不停 ， 辛苦 之相 也 ） ； 5 . 小腹 有 脂肪 ( 状是 一种 福寿之 相 ) 。 $ LOTOzf $',
 '中央 经济 工作 会议 1

### 停用词使用

In [ ]:
# 加载停用词
stoplist = [word.strip() for word in open(r'D:\NLP自然语言处理\text_classification\text classification\stop\stopword.txt', encoding='utf-8').readlines()]
stoplist

### 编码器处理文本标签

In [17]:
le = LabelEncoder()

y_train_le = le.fit_transform(y_train)
y_test_le = le.fit_transform(y_test)

y_test_le, y_train_le

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2], dtype=int64),
 array([1, 1, 1, ..., 2, 2, 2], dtype=int64))

In [18]:
# 文本数据转换成数据矩阵
# 词频矩阵
count = CountVectorizer(stop_words=stoplist, max_features=5000).fit(X_train_cut)
X_train_count = count.transform(X_train_cut)
X_test_count = count.transform(X_test_cut)

X_test_count, X_train_count


(<200x5000 sparse matrix of type '<class 'numpy.int64'>'
 	with 2829 stored elements in Compressed Sparse Row format>,
 <3305x5000 sparse matrix of type '<class 'numpy.int64'>'
 	with 54595 stored elements in Compressed Sparse Row format>)

### 算法模型
封装一个函数，提高复用率，使用是只需调用函数即可

In [21]:
# 用于储存所有算法的名字，准确率和所消耗的时间
import time
estimator_list, score_list, time_list = [], [], []


In [24]:
def get_text_classification(estimator, X, y, X_test, y_test):
    '''
    estimator: 分类器，必选参数
            X: 特征训练数据，必选参数
            y: 标签训练数据，必选参数
       X_test: 特征测试数据，必选参数
        y_tes: 标签测试数据，必选参数
       return: 返回值
           y_pred_model: 预测值
             classifier: 分类器名字
                  score: 准确率
                      t: 消耗的时间
                  matrix: 混淆矩阵
                  report: 分类评价函数
                       
    '''
    start = time.time()
    
    print('\n>>>算法正在启动，请稍候...')
    model = estimator
    
    print('\n>>>算法正在进行训练，请稍候...')
    model.fit(X, y)
    print(model)
    
    print('\n>>>算法正在进行预测，请稍候...')
    y_pred_model = model.predict(X_test)
    print(y_pred_model)
    
    print('\n>>>算法正在进行性能评估，请稍候...')
    score = metrics.accuracy_score(y_test, y_pred_model)
    matrix = metrics.confusion_matrix(y_test, y_pred_model)
    report = metrics.classification_report(y_test, y_pred_model)

    print('>>>准确率\n', score)
    print('\n>>>混淆矩阵\n', matrix)
    print('\n>>>召回率\n', report)
    print('>>>算法程序已经结束...')
    
    end = time.time()
    t = end - start
    print('\n>>>算法消耗时间为:', t, '秒\n')
    classifier = str(model).split('(')[0]
    
    return y_pred_model, classifier, score, round(t, 2), matrix, report
  

In [25]:
# K近邻
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=5)
result = get_text_classification(knc,X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])


>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
KNeighborsClassifier()

>>>算法正在进行预测，请稍候...
[1 0 2 1 2 2 1 1 2 1 1 1 1 1 2 1 1 2 0 1 1 1 1 1 0 1 2 0 1 1 0 2 2 2 2 2 0
 1 0 0 3 0 0 2 0 2 0 0 2 0 0 2 2 0 0 2 0 0 0 1 0 2 2 2 1 0 0 3 2 0 0 0 2 1
 0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 0 2 2 0 2 2 0 0 0 2 0 0 0 0 0 2 2 2 0 0 2 0
 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 2 0 0 0 2 0 0 2 0 0 0 2 2 2 0 0 2 2 0 2 2 1
 0 0 0 0 0 2 3 0 3 0 3 2 2 3 3 0 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2
 2 2 2 0 2 2 2 2 2 2 0 2 0 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.64

>>>混淆矩阵
 [[75  5 33  2]
 [ 6 20 12  0]
 [ 4  0 27  0]
 [ 3  0  7  6]]

>>>召回率
               precision    recall  f1-score   support

           0       0.85      0.65      0.74       115
           1       0.80      0.53      0.63        38
           2       0.34      0.87      0.49        31
           3       0.75      0.38      0.50        16

    accuracy                           0.64       200
   macro avg       0.69      0.61      0.59       200
weighted avg     

(None, None, None)

### 决策树

In [29]:
dtc = DecisionTreeClassifier()

result = get_text_classification(dtc, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])


>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
DecisionTreeClassifier()

>>>算法正在进行预测，请稍候...
[1 1 2 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 2 1 2 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 2 2 0 0 0 2 0 0 0 0 2 0
 2 0 0 0 0 2 0 2 2 2 0 0 0 0 0 0 3 3 0 0 1 1 0 0 0 0 0 0 0 0 2 0 0 2 0 2 0
 2 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 2 2 0 0 2 2 1 2
 2 0 0 0 0 2 3 1 3 2 3 3 3 3 3 0 3 2 2 2 3 2 1 2 2 2 2 2 2 2 1 2 2 2 2 2 3
 3 2 2 2 1 2 3 3 0 2 1 0 2 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.735

>>>混淆矩阵
 [[86  5 22  2]
 [ 1 31  5  1]
 [ 2  4 21  4]
 [ 1  1  5  9]]

>>>召回率
               precision    recall  f1-score   support

           0       0.96      0.75      0.84       115
           1       0.76      0.82      0.78        38
           2       0.40      0.68      0.50        31
           3       0.56      0.56      0.56        16

    accuracy                           0.73       200
   macro avg       0.67      0.70      0.67       200
weighted avg  

(None, None, None)

In [30]:
# 伯努利贝叶斯
bnb = BernoulliNB()

result = get_text_classification(bnb, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])



>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
BernoulliNB()

>>>算法正在进行预测，请稍候...
[1 1 2 1 0 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 0 1 2 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 2 0 0 0 2 0 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2 2 2 0 2
 0 0 0 0 0 0 3 2 3 0 2 2 0 0 3 2 3 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.85

>>>混淆矩阵
 [[104   1   9   1]
 [  2  32   4   0]
 [  1   0  30   0]
 [  8   0   4   4]]

>>>召回率
               precision    recall  f1-score   support

           0       0.90      0.90      0.90       115
           1       0.97      0.84      0.90        38
           2       0.64      0.97      0.77        31
           3       0.80      0.25      0.38        16

    accuracy                           0.85       200
   macro avg       0.83      0.74      0.74       200
weighted a

(None, None, None)

In [33]:
# 多项式贝叶斯
mnb = MultinomialNB()

result = get_text_classification(mnb, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])


>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
MultinomialNB()

>>>算法正在进行预测，请稍候...
[1 1 3 1 3 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 2 3 2 0 2
 0 0 0 0 0 3 3 2 3 3 3 2 3 3 3 3 3 0 0 0 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.915

>>>混淆矩阵
 [[107   0   5   3]
 [  0  34   2   2]
 [  0   0  31   0]
 [  3   0   2  11]]

>>>召回率
               precision    recall  f1-score   support

           0       0.97      0.93      0.95       115
           1       1.00      0.89      0.94        38
           2       0.78      1.00      0.87        31
           3       0.69      0.69      0.69        16

    accuracy                           0.92       200
   macro avg       0.86      0.88      0.86       200
weighte

(None, None, None)

In [39]:
# 高斯贝叶斯

gnb = GaussianNB()

result = get_text_classification(gnb, X_train_count.toarray(), y_train_le, X_test_count.toarray(), y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])


>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
GaussianNB()

>>>算法正在进行预测，请稍候...
[1 1 1 1 3 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 0 2 1 2 1 2 1
 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 0
 0 0 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2
 0 0 0 0 0 0 3 0 0 3 3 2 3 3 3 2 3 3 3 3 3 2 3 2 2 2 2 0 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 1 2 0 2 2 0 2 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.855

>>>混淆矩阵
 [[104   2   9   0]
 [  1  31   5   1]
 [  3   2  25   1]
 [  3   0   2  11]]

>>>召回率
               precision    recall  f1-score   support

           0       0.94      0.90      0.92       115
           1       0.89      0.82      0.85        38
           2       0.61      0.81      0.69        31
           3       0.85      0.69      0.76        16

    accuracy                           0.85       200
   macro avg       0.82      0.80      0.81       200
weighted a

(None, None, None)

In [40]:
# 逻辑回归

lg = LogisticRegression()

result = result = get_text_classification(lg, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])




>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
LogisticRegression()

>>>算法正在进行预测，请稍候...
[1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 1 0 2 0 1 0 2
 2 0 0 0 0 0 3 2 3 2 3 3 3 3 3 1 3 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 1 2 2 2 2 2 2 1 0 2 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.89

>>>混淆矩阵
 [[104   3   8   0]
 [  0  37   0   1]
 [  1   2  28   0]
 [  1   1   5   9]]

>>>召回率
               precision    recall  f1-score   support

           0       0.98      0.90      0.94       115
           1       0.86      0.97      0.91        38
           2       0.68      0.90      0.78        31
           3       0.90      0.56      0.69        16

    accuracy                           0.89       200
   macro avg       0.86      0.84      0.83       200
wei

(None, None, None)

In [43]:
# 支持向量机

svc = svm.SVC(kernel='sigmoid')

result = get_text_classification(svc, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])


>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
SVC(kernel='sigmoid')

>>>算法正在进行预测，请稍候...
[1 1 0 1 2 0 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 1 0 2 0 2 0 2
 2 0 0 0 0 0 3 2 3 2 3 3 3 3 3 0 3 0 0 0 3 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2
 2 2 2 1 2 2 3 2 2 2 2 0 2 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.865

>>>混淆矩阵
 [[104   3   8   0]
 [  2  33   3   0]
 [  2   1  27   1]
 [  5   0   2   9]]

>>>召回率
               precision    recall  f1-score   support

           0       0.92      0.90      0.91       115
           1       0.89      0.87      0.88        38
           2       0.68      0.87      0.76        31
           3       0.90      0.56      0.69        16

    accuracy                           0.86       200
   macro avg       0.85      0.80      0.81       200
w

(None, None, None)

In [45]:
# 集成学习算法

# 随机森林算法

rfc = RandomForestClassifier()

result = get_text_classification(rfc, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])


>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
RandomForestClassifier()

>>>算法正在进行预测，请稍候...
[1 1 2 1 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 2 1 2 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 2 0
 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 1 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 1 2 2 2 2 3 2
 3 0 0 0 0 1 3 2 3 2 3 3 3 3 3 0 3 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 3 2 0 2 2 2 2 2 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.81

>>>混淆矩阵
 [[93  3 15  4]
 [ 0 32  3  3]
 [ 2  0 28  1]
 [ 1  1  5  9]]

>>>召回率
               precision    recall  f1-score   support

           0       0.97      0.81      0.88       115
           1       0.89      0.84      0.86        38
           2       0.55      0.90      0.68        31
           3       0.53      0.56      0.55        16

    accuracy                           0.81       200
   macro avg       0.73      0.78      0.74       200
weighted avg   

(None, None, None)

In [47]:
# 自增强算法

abc = AdaBoostClassifier(n_estimators=100)


result = get_text_classification(abc, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])


>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
AdaBoostClassifier(n_estimators=100)

>>>算法正在进行预测，请稍候...
[1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 0 1 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 0 1 0
 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 1 2 1 3 1 2 3 2 3 2 0 3 1 1 1 3 2 1 2 1 2 2 0 2 2 1 1 2 1 1 2 3
 2 1 2 1 1 2 3 3 3 3 0 0 1 1 2]

>>>算法正在进行性能评估，请稍候...
>>>准确率
 0.59

>>>混淆矩阵
 [[64 51  0  0]
 [ 1 37  0  0]
 [ 3 11 12  5]
 [ 1  6  4  5]]

>>>召回率
               precision    recall  f1-score   support

           0       0.93      0.56      0.70       115
           1       0.35      0.97      0.52        38
           2       0.75      0.39      0.51        31
           3       0.50      0.31      0.38        16

    accuracy                           0.59       200
   macro avg       0.63      0.56      0.53       200
wei

(None, None, None)

In [60]:
xgb = xgboost.XGBClassifier()

result = get_text_classification(xgb, X_train_count, y_train_le, X_test_count, y_test_le)
estimator_list.append(result[1]), score_list.append(result[2]), time_list.append(result[3])
# model = xgb.fit(X_train_count, y_train)
# pre = model.predict(X_test_count)
# score = metrics.accuracy_score(y_test, pre)



>>>算法正在启动，请稍候...

>>>算法正在进行训练，请稍候...
[17:26:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1

(None, None, None)

In [61]:
df = pd.DataFrame()
df['分类器'] = estimator_list
df['准确率'] = score_list
df['消耗时间/s'] = time_list
df

,分类器,准确率,消耗时间/s
0,KNeighborsClassifier,0.640,0.04
1,DecisionTreeClassifier,0.415,0.03
2,DecisionTreeClassifier,0.400,0.02
3,DecisionTreeClassifier,0.735,0.18
4,BernoulliNB,0.850,0.01
5,MultinomialNB,0.915,0.01
6,MultinomialNB,0.915,0.01
7,MultinomialNB,0.915,0.01
8,MultinomialNB,0.915,0.01
9,GaussianNB,0.855,0.31
